# Olist E-Commerce KPI Analysis

This notebook contains an end-to-end SQL-based analysis of the Brazilian Olist e-commerce dataset.
The objective is to analyze revenue performance, customer behavior, payment methods, and retention patterns.


## 1. Data Loading

In this section, we load the Olist dataset files (customers, orders, and payments) into the notebook
and prepare them for SQL-based analysis.


In [1]:
import pandas as pd

df = pd.read_csv('olist_customers_dataset.csv')

# Print information about columns, data types, and non-null values
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None


In [2]:

df = pd.read_csv('olist_order_payments_dataset.csv')

# Print information about columns, data types, and non-null values
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB
None


In [3]:

df = pd.read_csv('olist_orders_dataset.csv')

# Print information about columns, data types, and non-null values
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None


## 2. Creating SQL Database

Here, we create a SQLite database from the CSV files to simulate a real-world relational database
environment and enable SQL querying.


In [4]:
import sqlite3
import pandas as pd

# Create SQLite database
conn = sqlite3.connect("olist.db")

# Load CSVs
customers = pd.read_csv("olist_customers_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")

# Write to SQL tables
customers.to_sql("customers", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)
payments.to_sql("payments", conn, if_exists="replace", index=False)

print("Database created successfully!")


Database created successfully!


## 3. Understanding the Data Schema

The analysis uses three main tables:

- customers → customer-level information (city, state)
- orders → order-level transactions
- payments → payment details for each order

Relationships:
- customers.customer_id → orders.customer_id
- orders.order_id → payments.order_id


## 4. Total Revenue

This KPI measures the total monetary value generated across all orders on the platform.
It helps understand overall business scale.


In [5]:
pd.read_sql("""
SELECT SUM(payment_value) AS total_revenue
FROM payments;
""", conn)


,total_revenue
0,1.600887e+07


## 5. Revenue by City

This analysis identifies the cities that generate the highest revenue,
helping prioritize regions for marketing and operational focus.


In [9]:
# top 5 cities by revenue
pd.read_sql("""
SELECT customers.customer_city, SUM(payments.payment_value) as total_revenue
FROM customers
INNER JOIN orders
ON customers.customer_id = orders.customer_id
INNER JOIN payments
ON orders.order_id = payments.order_id
GROUP BY customers.customer_city
ORDER BY total_revenue DESC
LIMIT 5
""", conn)

,customer_city,total_revenue
0,sao paulo,2203373.09
1,rio de janeiro,1161927.36
2,belo horizonte,421765.12
3,brasilia,354216.78
4,curitiba,247392.48


Insight:
São Paulo generates significantly higher total revenue compared to other cities, making it the platform’s primary revenue hub.

KPI Used:
Top Cities by Total Revenue (Task E)

Business Action:
Prioritize marketing budget, seller onboarding, and premium campaigns in São Paulo to maximize revenue growth and ROI.

## 6. Average Order Value (AOV) by City

AOV indicates how much customers spend per order on average in each city.
This helps differentiate between high-volume and high-value regions.


In [10]:
# AOV per city using
pd.read_sql("""
SELECT customers.customer_city, SUM(payments.payment_value) / COUNT(DISTINCT orders.order_id) AS average_order_revenue
FROM customers
INNER JOIN orders
ON customers.customer_id = orders.customer_id
INNER JOIN payments
ON orders.order_id = payments.order_id
GROUP BY customers.customer_city
ORDER BY average_order_revenue DESC
""", conn)

,customer_city,average_order_revenue
0,pianco,2324.99
1,nova esperanca do piria,2252.66
2,engenheiro navarro,2106.55
3,agrestina,2066.34
4,mariental,1867.85
...,...,...
4114,santo antonio do rio abaixo,24.23
4115,tamboara,24.09
4116,jenipapo de minas,22.58
4117,polo petroquimico de triunfo,20.70


Insight:
Piancó shows the highest average order value, indicating customers there tend to place higher-value orders.

KPI Used:
Average Order Value (AOV) by City (Task F)

Business Action:
Introduce loyalty rewards, targeted promotions, and premium product recommendations for customers in Piancó to maintain and increase high-value purchases.

## 6.1 Top 5 cities by order

This query tells us top 5 cities with respect to total orders being ordered


In [11]:
# top 5 cities by order

pd.read_sql("""
SELECT c.customer_city, COUNT(DISTINCT o.order_id) AS total_orders
FROM customers c
INNER JOIN orders o
ON c.customer_id = o.customer_id
GROUP BY c.customer_city
ORDER BY total_orders DESC
LIMIT 5
 """, conn)

,customer_city,total_orders
0,sao paulo,15540
1,rio de janeiro,6882
2,belo horizonte,2773
3,brasilia,2131
4,curitiba,1521


Insight:
São Paulo leads in total order volume, indicating the highest demand and platform activity.

KPI Used:
Top Cities by Number of Orders (Task G)

Business Action:
Strengthen logistics capacity, delivery partnerships, and consider localized warehouses or fulfillment centers in São Paulo to support high order volume efficiently.

## 7. Revenue by Payment Method

This section analyzes which payment methods contribute the most to total revenue,
providing insights into customer payment preferences.


In [12]:
# revenue by payment type
pd.read_sql("""
SELECT p.payment_type, SUM(p.payment_value) AS total_revenue
FROM payments p
GROUP BY p.payment_type
ORDER BY total_revenue DESC
""", conn)

,payment_type,total_revenue
0,credit_card,1.254208e+07
1,boleto,2.869361e+06
2,voucher,3.794369e+05
3,debit_card,2.179898e+05
4,not_defined,0.000000e+00


Insight:
Credit cards account for the majority of platform revenue, making them the dominant payment method.

KPI Used:
Revenue by Payment Method (Task H)

Business Action:
Optimize checkout UX for credit card payments, set credit cards as the default option, and negotiate better transaction fees with payment providers.

## 8. Repeat Customer Analysis

This KPI identifies cities with a higher concentration of repeat customers,
indicating stronger customer loyalty and retention potential.


In [13]:
# cities with highest number of customers
pd.read_sql("""
SELECT c.customer_city, COUNT(c.customer_id) AS customer_more_than_two_orders
FROM customers c
INNER JOIN orders o
ON c.customer_id = o.customer_id
GROUP BY c.customer_city
HAVING COUNT(o.order_id) >= 2
ORDER BY customer_more_than_two_orders DESC
""", conn)

,customer_city,customer_more_than_two_orders
0,sao paulo,15540
1,rio de janeiro,6882
2,belo horizonte,2773
3,brasilia,2131
4,curitiba,1521
...,...,...
2970,acaiaca,2
2971,abrantes,2
2972,abare,2
2973,abaira,2


Insight:
São Paulo has the highest concentration of repeat customers, indicating stronger customer loyalty in this city.

KPI Used:
Repeat Customer Concentration by City (Task I)

Business Action:
Launch targeted retention campaigns, loyalty programs, and personalized offers in São Paulo to further increase customer lifetime value.

## 9. Key Business Insights

- São Paulo is the primary revenue and demand hub.
- Certain cities show higher average order values, indicating premium purchasing behavior.
- Credit cards dominate revenue contribution.
- Repeat customers are concentrated in specific cities, highlighting retention opportunities.


## 10. Conclusion

This analysis demonstrates how SQL can be used to extract actionable business insights
from real-world e-commerce data. The results can support decision-making across marketing,
logistics, payment optimization, and customer retention strategies.
